In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [59]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import pyLDAvis.sklearn

import string

In [75]:
from sklearn.manifold import TSNE
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import os


In [5]:
df = pd.read_csv('LSQ3.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [178]:
df_filter = df[df['year'] > 2014]

In [179]:
df_filter['Question']

610        a) the details of the funds allocated to the S...
611        (a) the budgetary allocation of the welfare pr...
612        (a) whether the headquarters of the banks have...
615        (a) whether the Indian Railways is expediting ...
616        (a)the number of Beedi workers working in vari...
                                 ...                        
1794608    (a) whether the Railways has tied up with some...
1794610    (a) Whether the country is the largest recipie...
1794611    (a) whether the Government has asked the Reser...
1794613    Will the Minister of URBAN DEVELOPMENT be plea...
1794615    (a) whether the Government proposes to close a...
Name: Question, Length: 314250, dtype: object

In [180]:
nlp = spacy.load('en')

In [181]:
doc = nlp(df_filter['Question'][620])
spacy.displacy.render(doc, style = 'ent', jupyter=True)

In [182]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

In [183]:
review = str(" ".join([i.lemma_ for i in doc]))

In [184]:
doc = nlp(review)
spacy.displacy.render(doc, style='ent',jupyter=True)

In [185]:
for i in nlp(review):
    print(i,"=>",i.pos_)

( => PUNCT
a => X
) => PUNCT
whether => SCONJ
the => DET
Government => PROPN
have => AUX
allocate => ADJ
responsibility => NOUN
of => ADP
set => VERB
up => ADP
of => ADP
1000 => NUM
MW => PROPN
solar => ADJ
power => NOUN
plant => NOUN
under => ADP
the => DET
Jawaharlal => PROPN
Nehru => PROPN
National => PROPN
Solar => PROPN
Mission => PROPN
( => PUNCT
JNNSM => PROPN
) => PUNCT
to => ADP
NTPC => PROPN
VidyutVyapar => PROPN
Nigam => PROPN
Limited => PROPN
in => ADP
Telangana => PROPN
; => PUNCT
   => SPACE
( => PUNCT
b => X
) => PUNCT
if => SCONJ
so => ADV
, => PUNCT
the => DET
detail => NOUN
thereof => ADV
; => PUNCT
   => SPACE
( => PUNCT
c => NOUN
) => PUNCT
whether => SCONJ
the => DET
land => NOUN
have => AUX
be => AUX
acquire => NOUN
for => ADP
the => DET
purpose => NOUN
, => PUNCT
if => SCONJ
so => ADV
, => PUNCT
the => DET
detail => NOUN
thereof => ADV
and => CCONJ
the => DET
time => NOUN
by => ADP
which => PRON
the => DET
project => NOUN
be => AUX
likely => ADJ
to => PART
be => 

In [186]:
custom_stop_words = frozenset(['government', 'whether', 'details', 'thereof', 'reasons', 'action',
                            'therefor', 'step', 'year', 'years', 'state', 'india', 'indian','said', 'received',
                               'steps', 'actions', 'regard', 'taken', 'propose', 'thereon', 'wise',
                              'country', 'proposes', 'proposed', 'act', 'thereto'])

In [187]:
from sklearn.feature_extraction import text 

stop_words = text.ENGLISH_STOP_WORDS.union(custom_stop_words)

In [188]:
parser = English()

def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [189]:
df_filter['Question'] = df_filter['Question'].apply(spacy_tokenizer)

In [190]:
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words= stop_words, 
                             lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')

data_vectorized = vectorizer.fit_transform(df_filter["Question"])

In [191]:
NUM_TOPICS = 10

In [192]:
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, 
                                max_iter=10, 
                                learning_method='online',
                                verbose=True)

data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [193]:
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

In [194]:
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
[('research', 16432.798899112127), ('education', 15876.108545037723), ('committee', 13509.912123203976), ('countries', 12981.302278475308), ('energy', 12525.364435254822), ('policy', 12510.915594845681), ('likely', 10984.430784024593), ('study', 10871.850312459905), ('schools', 10701.168187095142), ('report', 9660.143185358127)]
Topic 1:
[('projects', 35380.696714129524), ('time', 28361.205060741562), ('likely', 26261.472012292375), ('project', 21909.492422169347), ('including', 19617.384618449603), ('pradesh', 18680.178277943236), ('status', 18245.03490932582), ('development', 15757.895892351766), ('national', 15139.198784677452), ('funds', 15131.430992662325)]
Topic 2:
[('production', 18514.575278326898), ('farmers', 18425.67360682538), ('food', 13844.29968839183), ('sector', 12792.496229743352), ('products', 10969.804906899122), ('current', 10652.470558024086), ('industry', 9825.368255876641), ('domestic', 9522.487787299271), ('increase', 9147.025226935228), ('pr

In [195]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
6     -141.307419 -213.730515       1        1  15.625128
0     -211.072922  -33.629295       2        1  12.627899
1      -19.594727  -82.753448       3        1  12.310997
2      210.655930  165.237289       4        1  11.454306
3       43.453976 -270.044525       5        1  10.337227
8      280.453949  -14.862207       6        1   9.051136
4       25.898289  221.525925       7        1   8.765197
9      166.155441 -146.355804       8        1   7.302920
7      -96.696907   97.764999       9        1   6.506060
5       89.019127   34.192261      10        1   6.019129, topic_info=      Category          Freq       Term         Total  loglift  logprob
23903  Default  58936.000000     scheme  58936.000000  30.0000  30.0000
16776  Default  41990.000000   minister  41990.000000  29.0000  29.0000
20335  Default  39092.000000    pleased  39092.000000  28.0000  28.0000
10413  Default  58088.000000      funds  58088.000000  27.0000  27.0000
21150  Default  35863.000000   projects  35863.000000  26.0000  26.0000
...        ...           ...        ...           ...      ...      ...
830    Topic10   3217.940016        air  13475.182390   1.3781  -5.1582
20335  Topic10   3788.898747    pleased  39092.546299   0.4764  -4.9948
26186  Topic10   2656.506330     supply   9039.507093   1.5856  -5.3499
21112  Topic10   2745.088478  programme  18424.726680   0.9063  -5.3171
17999  Topic10   2715.941233        new  33566.941154   0.2958  -5.3278

[537 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
101        7  0.999481    aadhaar
262        8  0.999840  accidents
627        8  0.803285    affairs
627       10  0.196498    affairs
671        5  0.999449  aforesaid
...      ...       ...        ...
29938      3  0.999890       yoga
29942      1  0.518334     yojana
29942      9  0.481553     yojana
29955      8  0.999555     youths
29990      7  0.999854       zone

[868 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 1, 2, 3, 4, 9, 5, 10, 8, 6])

In [196]:
def spacy_bigram_tokenizer(phrase):
    doc = parser(phrase) # create spacy object
    token_not_noun = []
    notnoun_noun_list = []
    noun = ""

    for item in doc:
        if item.pos_ != "NOUN": # separate nouns and not nouns
            token_not_noun.append(item.text)
        if item.pos_ == "NOUN":
            noun = item.text
        
        for notnoun in token_not_noun:
            notnoun_noun_list.append(notnoun + " " + noun)

    return " ".join([i for i in notnoun_noun_list])

In [197]:
bivectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                               stop_words= stop_words, 
                               lowercase=True, ngram_range=(1,2))

bigram_vectorized = bivectorizer.fit_transform(df_filter['Question'])

In [198]:
bi_lda = LatentDirichletAllocation(n_components=NUM_TOPICS, 
                                   max_iter=10, 
                                   learning_method='online',
                                   verbose=True)

data_bi_lda = bi_lda.fit_transform(bigram_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [199]:
print("Bi-LDA Model:")
selected_topics(bi_lda, bivectorizer)

Bi-LDA Model:
Topic 0:
[('power', 21087.840231522536), ('technology', 13797.768458482997), ('banks', 11890.158047915404), ('bank', 10173.335944082592), ('plants', 9582.30637901364), ('research', 9216.868398419105), ('information', 8489.215133001735), ('set', 6331.570573468725), ('sector', 6130.023815090609), ('telecom', 5340.122797125214)]
Topic 1:
[('railways', 23162.70090090781), ('water', 22160.908986944145), ('sector', 13356.881309617474), ('railway', 12765.023556637663), ('investment', 8422.07809686821), ('foreign', 8238.132133659581), ('stations', 7325.998920885796), ('new', 7265.466452664206), ('policy', 6721.018180690115), ('drinking', 6573.442724292874)]
Topic 2:
[('farmers', 19395.341420400957), ('current', 13023.338987631452), ('production', 10520.002674914736), ('sector', 9981.85123515431), ('scheme', 8975.800085141196), ('energy', 8623.6025255358), ('including', 8433.396079887994), ('states', 8383.942066814383), ('agricultural', 7806.039201277291), ('industry', 7421.462627

In [200]:
bi_dash = pyLDAvis.sklearn.prepare(bi_lda, bigram_vectorized, bivectorizer, mds='tsne')
bi_dash

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
7      22.605188 -55.378010       1        1  13.633661
8      56.695572  49.855049       2        1  11.488956
2       5.192758  92.106750       3        1  10.564871
9       5.123953  42.330551       4        1  10.138684
6     -48.789169  60.469818       5        1   9.695195
4      18.585297  -4.842831       6        1   9.632737
1      69.217628  -7.324473       7        1   9.425071
0     -78.741676   5.546642       8        1   9.143393
3     -34.930794 -44.644283       9        1   8.442788
5     -28.987955   7.128065      10        1   7.834644, topic_info=       Category          Freq         Term          Total  loglift  logprob
221874  Default  56924.000000        funds   56924.000000  30.0000  30.0000
418878  Default  35437.000000     projects   35437.000000  29.0000  29.0000
81406   Default  24810.000000        cases   24810.000000  28.0000  28.0000
106244  Default  28588.000000    companies   28588.000000  27.0000  27.0000
240212  Default  27594.000000       health   27594.000000  26.0000  26.0000
...         ...           ...          ...            ...      ...      ...
127928  Topic10   4484.732696    countries   18837.766300   1.1114  -5.7932
526361  Topic10   3402.434375       survey    8653.290453   1.6132  -6.0694
37823   Topic10   3127.203366       andhra    6670.342502   1.7891  -6.1538
355235  Topic10   3691.127421       number  103570.286157  -0.7877  -5.9880
153770  Topic10   3396.947293  development   43430.492626  -0.0017  -6.0710

[616 rows x 6 columns], token_table=        Topic      Freq              Term
term                                     
2518        3  0.129246                15
2518        4  0.031057                15
2518        5  0.050886                15
2518        8  0.021023                15
2518        9  0.048497                15
...       ...       ...               ...
582191      8  0.108330             world
582191      9  0.035664             world
582228      1  0.999426        world bank
583523      6  0.999838              yoga
583585      6  0.999782  yoga naturopathy

[1211 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 9, 3, 10, 7, 5, 2, 1, 4, 6])